## Train and evaluate model using Scikit-learn and Keras wrapper 

In [10]:
import tensorflow as tf
from aMNWtModel import AMNWtModel


seed = 10
# np.random.seed(seed=seed)  
# tf.random.set_seed(seed)

# dataset_file = "./Dataset/IJN1463_EXP_UB_Anne.npz"
# objective=['BIOMASS_KT2440_WT3']

# dataset_file = "./Dataset/IJN1463_10_UB.npz"
# objective=['BIOMASS_KT2440_WT3']

# dataset_file = "./Dataset/e_coli_core_UB_100.npz"
# objective=['BIOMASS_Ecoli_core_w_GAM']
# epoch = 200
# batch_size = 7


# dataset_file = "./Dataset/biolog_iML1515_EXP_UB.npz"
# objective=['BIOMASS_Ec_iML1515_core_75p37M']
# epoch = 20
# batch_size = 30

dataset_file = "./Dataset/biolog_iML1515_new_UB.npz"
objective=['BIOMASS_Ec_iML1515_core_75p37M']
epoch = 20
batch_size = 30


print("---------------------------------------- model ----------------------------------------")
model = AMNWtModel(dataset_file=dataset_file, 
                   objective=objective,
                   timestep=4,
                #    n_hidden=1, 
                   hidden_dim=50,
                   epochs=50, 
                   verbose=True,
                   batch_size=7)
model.printout()

# Preprocessing
from sklearn.preprocessing import MinMaxScaler,StandardScaler,MaxAbsScaler 
from tools import MaxScaler
scaler= MaxScaler()
model.train_test_split(test_size=0.1, random_state=seed)
model.preprocess(scaler)
model.preprocessing_for_specific_model()

---------------------------------------- model ----------------------------------------
number of metabolites:  1877
filtered measurements size:  1
dataset file: ./Dataset/biolog_iML1515_new_UB.npz
model type: AMNWt
model medium bound: UB
timestep: 4
training set size (17400, 430) (17400, 1)
training epochs: 50
training regression: True
training batch size: 7
training validation iter: 0
training early stopping: False


In [11]:
# cross validation
from sklearn.metrics import make_scorer
from sklearn.model_selection import KFold, cross_validate
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

estimator= KerasRegressor(build_fn=model.build_model, 
                          epochs=epoch, 
                          batch_size=batch_size, 
                          verbose=0)

scoring = {"loss_constraint":make_scorer(model.loss_constraint),
           "mse":make_scorer(model.mse),
           "R2":make_scorer(model.R2),
           }

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
fit_params = {'callbacks': [callback]}
fit_params = {}

kfold= KFold(n_splits=5,shuffle=True, random_state=seed)

results=cross_validate(estimator, 
                       model.X_train, 
                       model.Y_train, 
                       cv=kfold, 
                       n_jobs=5, 
                       scoring=scoring, 
                       fit_params=fit_params,
                       return_train_score=True)
results

2023-09-21 14:39:23.829147: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-21 14:39:23.830570: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 5. Tune using inter_op_parallelism_threads for best performance.
2023-09-21 14:39:24.091534: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-21 14:39:24.092926: I tensorflow/core/common_runtime/process_util.cc:

{'fit_time': array([4080.3157227 , 4080.54561496, 4076.42844009, 4087.82974768,
        4065.37598562]),
 'score_time': array([27.87980247, 29.91382384, 20.64488554, 27.45808148, 33.03862095]),
 'test_loss_constraint': array([0.00536525, 0.01075396, 0.00571801, 0.00593194, 0.00590252]),
 'train_loss_constraint': array([0.00546597, 0.01086891, 0.00557294, 0.00594837, 0.00586356]),
 'test_mse': array([0.38555504, 0.52376219, 0.33764069, 0.34447866, 0.46628941]),
 'train_mse': array([0.38361359, 0.5343125 , 0.33439099, 0.34913444, 0.46459096]),
 'test_R2': array([-2.66452905, -3.9376391 , -2.25171746, -2.36596027, -3.54783449]),
 'train_R2': array([-2.70585956, -4.14268876, -2.21163967, -2.34017798, -3.46586   ])}

In [12]:
AMNWt_model = model.build_model()
history = AMNWt_model.fit(model.X_train, model.Y_train, epochs=epoch, batch_size=batch_size, verbose=0)

print("R2 :", model.R2(model.Y_train, AMNWt_model.predict(model.X_train)))
print("Q2 :", model.R2(model.Y_test, AMNWt_model.predict(model.X_test)))

R2 : -2.902497158588206
Q2 : -3.2142680727890784


## Search for hyperparameters

In [4]:
from sklearn.model_selection import RandomizedSearchCV

estimator= KerasRegressor(build_fn=model.build_model, 
                          epochs=epoch, 
                          batch_size=batch_size, 
                          verbose=0)

distributions = dict(batch_size=[7,20],
                     nb_epoch=[2,100],
                    #  hidden_dim=[1,2],
                     )

scoring = {"loss_constraint":make_scorer(model.loss_constraint),
           "mse":make_scorer(model.mse),
           "R2":make_scorer(model.R2),
           }

clf = RandomizedSearchCV(estimator, distributions, random_state=0)
search = clf.fit(model.X_test, model.Y_test)

/home/agiralt/anaconda3/envs/AMN/lib/python3.9/site-packages/sklearn/model_selection/_search.py:285: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


## Save and load the model

In [5]:
import tensorflow as tf
from aMNWtModel import AMNWtModel


seed = 10
tf.random.set_seed(seed)
dataset_file = "./Dataset/e_coli_core_UB_100.npz"
objective=['BIOMASS_Ecoli_core_w_GAM']

# Dataset plus model structure
print("---------------------------------------- model ----------------------------------------")
model = AMNWtModel(dataset_file=dataset_file, 
                   objective=objective,
                   timestep=4,
                   hidden_dim=50,
                   epochs=50, 
                   verbose=True,
                   batch_size=7)
model.printout()

# Preprocessing
from sklearn.preprocessing import MinMaxScaler,StandardScaler,MaxAbsScaler 
from tools import MaxScaler
scaler= MaxScaler()
model.train_test_split(test_size=0.1, random_state=seed)
model.preprocess(scaler)
model.preprocessing_for_specific_model()

batch_size = 7

# Construct and train an AMNWt model
AMNWt_model = model.build_model()
history = AMNWt_model.fit(model.X_train, model.Y_train, epochs=200, batch_size=batch_size, verbose=0)

print("R2 :", model.R2(model.Y_train, AMNWt_model.predict(model.X_train)))
print("Q2 :", model.R2(model.Y_test, AMNWt_model.predict(model.X_test)))

---------------------------------------- model ----------------------------------------
number of metabolites:  72
filtered measurements size:  1
dataset file: ./Dataset/e_coli_core_UB_100.npz
model type: AMNWt
model medium bound: UB
timestep: 4
training set size (100, 20) (100, 1)
training epochs: 50
training regression: True
training batch size: 7
training validation iter: 0
training early stopping: False
R2 : 0.9512388023070849
Q2 : 0.8899013443185595


In [6]:
from aMNWtModel import RNNCell
from tools import my_mse

seed = 10
tf.random.set_seed(seed)

# Recreate new model from config file, compile and train it. First test on config.
config = AMNWt_model.get_config()
AMNWt_model_= tf.keras.Model.from_config(config, custom_objects={"RNNCell":RNNCell})
AMNWt_model_.compile(loss=my_mse,optimizer='adam',metrics=[my_mse])
history = AMNWt_model_.fit(model.X_train, model.Y_train, epochs=200, batch_size=batch_size, verbose=0)

print("R2 :", model.R2(model.Y_train, AMNWt_model_.predict(model.X_train)))
print("Q2 :", model.R2(model.Y_test, AMNWt_model_.predict(model.X_test)))

R2 : 0.9512388023070849
Q2 : 0.8899013443185595


In [7]:
# Save the train AMNWt_model
model_file = "Models/test_saving_model.keras"
tf.keras.models.save_model(AMNWt_model,model_file, overwrite=True, save_format=None, save_traces=True)

In [8]:
from tools import my_mse
from aMNWtModel import RNNCell

AMNWt_model_ = tf.keras.models.load_model(model_file, custom_objects={"RNNCell":RNNCell,
                                                                               "my_mse":my_mse})

history = AMNWt_model_.fit(model.X_train, model.Y_train, epochs=200, batch_size=batch_size, verbose=0)
print("R2 :", model.R2(model.Y_train, AMNWt_model_.predict(model.X_train)))
print("Q2 :", model.R2(model.Y_test, AMNWt_model_.predict(model.X_test)))

R2 : 0.9819044696340576
Q2 : 0.9144270593047028
